In [7]:
import MetaTrader5 as mt5
import pandas as pd
import logging
import traceback

# Set up logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("backtest_debug.log", mode='w')
    ]
)

def initialize_mt5(server, login, password):
    try:
        if not mt5.initialize():
            raise Exception(f"MT5 initialization failed: {mt5.last_error()}")
        if not mt5.login(login, password=password, server=server):
            raise Exception(f"Login failed: {mt5.last_error()}")
        logging.info("MT5 initialized and logged in successfully.")
    except Exception as e:
        logging.error("Error during MT5 initialization")
        logging.error(traceback.format_exc())
        raise e

def get_historical_data(symbol, timeframe, count=1000):
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
        if rates is None:
            raise Exception(f"Failed to retrieve rates: {mt5.last_error()}")
        data = pd.DataFrame(rates)
        data['time'] = pd.to_datetime(data['time'], unit='s')
        logging.info("Historical data retrieved successfully.")
        return data
    except Exception as e:
        logging.error("Error retrieving historical data")
        logging.error(traceback.format_exc())
        raise e

def backtest(data, balance=10000, risk_per_trade=0.05, reward_per_trade=0.10):
    results = []
    account_balance = balance

    try:
        for i in range(1, len(data)):
            entry_price = data['close'][i - 1]
            entry_time = data['time'][i]
            entry_day = entry_time.strftime("%A")

            if data['close'][i] < entry_price:  # Entry signal
                sl_bdt = account_balance * risk_per_trade
                tp_bdt = account_balance * reward_per_trade
                price_per_bdt = entry_price / account_balance

                sl = entry_price - (sl_bdt * price_per_bdt)
                tp = entry_price + (tp_bdt * price_per_bdt)

                for j in range(i, len(data)):
                    current_price = data['close'][j]
                    exit_time = data['time'][j]

                    if current_price >= tp:
                        profit_bdt = tp_bdt
                        account_balance += profit_bdt
                        results.append([
                            entry_time, exit_time, "TP", entry_day,
                            round(profit_bdt, 2), round(account_balance, 2)
                        ])
                        break
                    elif current_price <= sl:
                        loss_bdt = -sl_bdt
                        account_balance += loss_bdt
                        results.append([
                            entry_time, exit_time, "SL", entry_day,
                            round(loss_bdt, 2), round(account_balance, 2)
                        ])
                        break

        return results

    except Exception as e:
        logging.error("Error during backtesting")
        logging.error(traceback.format_exc())
        raise e

def calculate_statistics(results):
    total_trades = len(results)
    wins = sum(1 for trade in results if trade[2] == "TP")
    losses = sum(1 for trade in results if trade[2] == "SL")
    win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
    total_profit = sum(trade[4] for trade in results)
    avg_profit_per_trade = total_profit / total_trades if total_trades > 0 else 0

    stats = {
        "Total Trades": total_trades,
        "Wins": wins,
        "Losses": losses,
        "Win Rate (%)": round(win_rate, 2),
        "Average PnL (BDT)": round(avg_profit_per_trade, 2),
        "Net Profit (BDT)": round(total_profit, 2),
    }
    return stats

def main():
    server = "Exness-MT5Trial6"
    login = 240355464
    password = "23Dec&1997"
    symbol = "BTCUSDm"
    timeframe = mt5.TIMEFRAME_M5

    try:
        initialize_mt5(server, login, password)
        data = get_historical_data(symbol, timeframe, count=1000)
        results = backtest(data)
        
        results_df = pd.DataFrame(results, columns=[
            "Entry Time", "Exit Time", "TP/SL", "Day", "Amount (BDT)", "Account Balance (BDT)"
        ])
        print("\nBacktest Trade Details:")
        print(results_df)

        stats = calculate_statistics(results)
        stats_df = pd.DataFrame([stats])
        print("\nTrade Statistics:")
        print(stats_df)

    except Exception as e:
        logging.error("An error occurred in the main routine")
        logging.error(traceback.format_exc())
    finally:
        mt5.shutdown()
        logging.info("MT5 shutdown completed.")

if __name__ == "__main__":
    main()


2025-04-12 16:14:01,170 [INFO] MT5 initialized and logged in successfully.
2025-04-12 16:14:01,173 [INFO] Historical data retrieved successfully.
2025-04-12 16:14:03,935 [INFO] MT5 shutdown completed.



Backtest Trade Details:
            Entry Time           Exit Time TP/SL        Day  Amount (BDT)  \
0  2025-04-08 23:15:00 2025-04-11 18:00:00    TP    Tuesday       1000.00   
1  2025-04-08 23:25:00 2025-04-11 18:00:00    TP    Tuesday       1100.00   
2  2025-04-08 23:40:00 2025-04-11 18:00:00    TP    Tuesday       1210.00   
3  2025-04-08 23:50:00 2025-04-11 18:00:00    TP    Tuesday       1331.00   
4  2025-04-08 23:55:00 2025-04-11 17:40:00    TP    Tuesday       1464.10   
..                 ...                 ...   ...        ...           ...   
60 2025-04-09 23:15:00 2025-04-10 15:45:00    SL  Wednesday     -12593.80   
61 2025-04-09 23:20:00 2025-04-10 15:45:00    SL  Wednesday     -11964.11   
62 2025-04-09 23:25:00 2025-04-10 15:45:00    SL  Wednesday     -11365.90   
63 2025-04-09 23:35:00 2025-04-10 15:45:00    SL  Wednesday     -10797.61   
64 2025-04-09 23:45:00 2025-04-10 15:45:00    SL  Wednesday     -10257.73   

    Account Balance (BDT)  
0                11000